In [2]:
from utils import *

[nltk_data] Downloading package punkt to /Users/rfd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-05-08 14:29:30.332476: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train_df_original, val_df_original = get_processed_data(dev=False)

In [10]:
train_df_original.shape


(2560, 7)

In [9]:
val_df_original.shape

(640, 7)

In [5]:
class CustomPropagandaDataset_vanilla(Dataset):
    def __init__(self,df,dataset='prop'):
        
        tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
        
        if dataset == 'prop':
            target_col = 'propaganda'
            text_col = 'original_sentence_no_tags'
            
            self.labels=torch.tensor([label for label in df[target_col]])

            
        if dataset == 'snip':
            label_to_id = {'flag_waving': 0, 'exaggeration,minimisation': 1, 'causal_oversimplification': 2, 'name_calling,labeling': 3, 'repetition': 4, 'doubt': 5, 'not_propaganda': 6, 'loaded_language': 7, 'appeal_to_fear_prejudice': 8}
            
            target_col = 'snippet_label'
            text_col = 'snippet_original'
            self.labels=torch.tensor([label_to_id[label] for label in df[target_col]])
            
        self.texts=[tokenizer(text,padding='max_length',max_length=200,truncation=True,return_tensors="pt") for text in df[text_col]]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self,idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self,idx):
        return self.texts[idx]

    def __getitem__(self,idx):
        batch_texts=self.get_batch_texts(idx)
        batch_y=self.get_batch_labels(idx)

        return batch_texts,batch_y


def prepare_inputs(input1,label,device):
  label=label.to(device)
  mask=input1['attention_mask'].to(device)
  input_id=input1['input_ids'].squeeze(1).to(device)
  return (input_id,mask,label)

In [6]:
train_data = CustomPropagandaDataset_vanilla(train_df_original)
val_data = CustomPropagandaDataset_vanilla(val_df_original)

train_dataloader=torch.utils.data.DataLoader(train_data,batch_size=64,shuffle=True)
val_dataloader=torch.utils.data.DataLoader(val_data,batch_size=64)


In [7]:
train_data[0][1]

array(0)

In [8]:
for i in train_dataloader:
    print(len(i[1]))
    break

64
